## LABORATORIO : IA - AGENTES DE VENTAS

proyecto con el cual se realiza una arquitectura agentica para automatizar procesos de agentes de ventas como el envio de correos electronicos

herramientas:
* sendGrid -> para enviar correos 
* openIA SDK -> framework de desarrollo
* gemini -> LLM
* asycio -> programacion "asincrona" python

In [41]:
#librerias
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool, OpenAIChatCompletionsModel, input_guardrail, GuardrailFunctionOutput
#from agents.extensions.models.litellm_model import LitellmModel # necesita vertex
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio
from openai import AsyncOpenAI
from pydantic import BaseModel


In [3]:
#cargar keys
load_dotenv(override=True)

True

##### Instrucciones del sistema

Para openIa SDK las instrucciones del sistema corresponden a los conocidos system prompts, los cuales tienen como objetivo "configurar" o adecuar el modelo LLM con un "estado" inicial 

In [15]:
#prompt del sistema
instructions1 = "Eres un agente de ventas que trabaja para Fivok Cloud Soluctions, \
    una empresa que ofrece servicios de arquitectura en la nuve, data BI y consultoria de software\
    impulsada por IA. Redactas correos electrónicos en frío profesionales y serios. Responde solo con la estructura del correo"

instructions2 = "Eres un agente de ventas con sentido del humor y atractivo \
    que trabaja para Fivok Cloud Soluctions, una empresa que ofrece servicios de arquitectura en la nuve, data BI\
    y consultoria de software, impulsada por IA. \
    Redactas correos electrónicos en frío ingeniosos y atractivos que probablemente obtengan respuesta. Responde solo con la estructura del correo"

instructions3 = "Eres un agente de ventas muy activo que trabaja para Fivok Cloud Soluctions, \
    una empresa que ofrece servicios de arquitectura en la nuve, data BI y consultoria de software\
    impulsada por IA. Redactas correos electrónicos en frío concisos y directos. Responde solo con la estructura del correo"

In [4]:
#configuracion de agentes usuando Gemini
# ---- Gemini api key y url base
gemini_api_key = os.getenv('GEMINI_API_KEY')
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"

# ---- Gemini modelos
gemini_model_flash_20 = "gemini-2.0-flash"
gemini_model_flash_25 = "gemini-2.5-flash"

# ---- lite objecto para uso de modelos non_openIA
# ----- para opnenIA[litellmModel] y Gemini se necesita vertex
#lite_llm_model_20 = LitellmModel(model=gemini_model_flash_20, api_key=gemini_api_key)
#lite_llm_model_25 = LitellmModel(model=gemini_model_flash_25, api_key=gemini_api_key)
gemini_client = AsyncOpenAI(base_url=GEMINI_BASE_URL, api_key=gemini_api_key)
gemini_model_agent1 = OpenAIChatCompletionsModel(model=gemini_model_flash_20, openai_client=gemini_client)
gemini_model_agent2 = OpenAIChatCompletionsModel(model=gemini_model_flash_25, openai_client=gemini_client)

In [16]:


# ---- Agentes de ventas
sales_agent1 = Agent(
    name = "Agente de ventas profesional",
    instructions = instructions1,
    model = gemini_model_agent1
)

sales_agent2 = Agent(
    name = "Agente de ventas atractivo",
    instructions = instructions2,
    model = gemini_model_agent2
)

In [27]:
# obtener corrutina en tipo stremead
result = Runner.run_streamed(sales_agent1, input="Escribe un correo electrónico de ventas en frío")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

Asunto: Impulsa tu negocio con soluciones en la nube y IA de Fivok Cloud Solutions

Estimado/a [Nombre del contacto],

Espero que este correo le encuentre bien.

Mi nombre es [Tu nombre] y soy [Tu cargo] en Fivok Cloud Solutions. Nos especializamos en ayudar a empresas como la suya a optimizar sus operaciones y alcanzar sus objetivos de negocio mediante soluciones innovadoras en la nube, análisis de datos (BI) y consultoría de software impulsada por IA.

[Menciona un punto específico sobre la empresa del contacto, demostrando que has investigado. Por ejemplo: "He notado que [Nombre de la empresa] está expandiendo su presencia en [Mercado/Región]". O: "He leído con interés su reciente iniciativa en [Área específica]".]

En Fivok Cloud Solutions, ofrecemos:

*   **Arquitectura en la Nube:** Diseñamos e implementamos soluciones en la nube escalables, seguras y rentables, adaptadas a sus necesidades específicas.
*   **Data BI:** Transformamos sus datos en información valiosa que impulsa la

In [27]:
message = "Escribe un correo electrónico de ventas en frío"

# dos correos de forma asincrona: dos tareas diferentes al mismo tiempo
with trace("Correos electrónicos fríos en paralelo"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
    )

outputs = [result.final_output for result in results]




In [28]:

# Imprimir cada respuesta indexada para identificar cuál corresponde a cada agente
for i, output in enumerate(outputs, 1):
    print(f"Respuesta Agente {i}:")
    print(output)
    print("\n" + "="*50 + "\n")

# También mostrar información adicional
print(f"Total de respuestas: {len(outputs)}")
print(f"Agente 1: {sales_agent1.name}")
print(f"Agente 2: {sales_agent2.name}")

Respuesta Agente 1:
**Asunto:** [Asunto breve y llamativo que resalte un beneficio clave o un problema que resuelves]

**Saludo:**

**Introducción:**

*   Presentación breve de Fivok Cloud Solutions.
*   Mencionar cómo encontraste a la empresa del destinatario (ej: investigación, referencia mutua, etc.).
*   Frase personalizada que demuestre que has investigado a la empresa y entiendes sus desafíos/oportunidades.

**Problema/Oportunidad:**

*   Identificar un problema común que enfrentan empresas como la del destinatario (relacionado con la nube, datos, software, etc.).
*   Presentar una oportunidad que podrían aprovechar si mejoran su infraestructura, análisis de datos o software.
*   (Opcional) Mencionar brevemente cómo este problema/oportunidad impacta sus resultados.

**Solución de Fivok Cloud Solutions:**

*   Describir brevemente cómo los servicios de Fivok Cloud Solutions (arquitectura en la nube, data BI, consultoría de software impulsada por IA) pueden abordar el problema/apro

In [30]:
#agente selector de correos
sales_picker = Agent(
    name="sales_picker",
    instructions="Elige el mejor correo electrónico de ventas en frío entre las opciones disponibles. \
        Imagina que eres un cliente y elige el que probablemente te responda. \
        No des explicaciones; responde solo con el correo electrónico seleccionado.",
    model = gemini_model_agent2
)

In [35]:
message = "Escribe un correo electrónico de ventas en frío"

with trace("Selección del equipo de ventas"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
    )
    outputs = [result.final_output for result in results]

    emails = "Emails de ventas en frío:\n\n".join(outputs)

    best = await Runner.run(sales_picker, emails)

    print(f"El mejor email de ventas:\n{best.final_output}")

El mejor email de ventas:
**Asunto:** ¿Tu nube te está pesando o elevando? ☁️

Hola [Nombre del Contacto],

Sé que tu bandeja de entrada es como un universo paralelo de mensajes, así que seré tu estrella fugaz: directo al punto y, espero, dejando una buena impresión. Mi nombre es [Tu Nombre] y, aunque no soy un genio con la lámpara, tengo algo mejor: Fivok Cloud Soluctions.

Aquí en Fivok, nos hemos dado cuenta de que muchas empresas están 'en la nube', pero pocas están realmente 'en el cielo' de la eficiencia y la innovación. ¿Te suena? Gestionar una arquitectura compleja, descifrar tus datos o lanzar software que realmente funcione... puede ser un verdadero rompecabezas.

Nosotros somos los maestros de ese rompecabezas. Con nuestra magia de arquitectura en la nube, inteligencia de datos (BI) para sacar el máximo jugo a tu información, y consultoría de software, todo impulsado por IA, transformamos tus desafíos digitales en tus mayores ventajas competitivas.

Imagina: tu infraestructu

## Adicionando Herramientas

para que una funcion se comporte o se pueda traducir como una herramienta para el modelo, se debe colocar la anotacion `@function_tool`


In [11]:
@function_tool #decorador o anotacion que construye el json con la estructura de una tool
def send_email(body: str):
    """ Envía un correo electrónico con el cuerpo indicado a todos los clientes potenciales de ventas. """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("ing.cristian.roac@gmail.com")  # Cambiar a tu remitente verificado
    to_email = To("ing.cristian.roac@gmail.com")  # Cambiar a sureceptor
    content = Content("text/plain", body)
    mail = Mail(from_email, to_email, "Email de ventas", content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [12]:
send_email

FunctionTool(name='send_email', description='Envía un correo electrónico con el cuerpo indicado a todos los clientes potenciales de ventas.', params_json_schema={'properties': {'body': {'title': 'Body', 'type': 'string'}}, 'required': ['body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001A8BA919B20>, strict_json_schema=True, is_enabled=True)

## Agentes como herramientas
Tambien podemos convertir un objeto Agent() en una tool para que otro llm decida si debe usarse en un flujo

In [31]:
description = "Escribe un email de ventas en frío"

tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)

In [32]:
# definicion de todas las herramientas
tools = [tool1, tool2, send_email]

### Coordinador de flujo

Ahora se crea un gerente de ventas que "decide" que herramienta o agente invocar

In [47]:
instructions ="Eres gerente de ventas y trabajas para ComplAI. \
Utilizas las herramientas que te proporcionamos para generar correos electrónicos de ventas en frío. \
Nunca generas correos electrónicos de ventas tú mismo; siempre usas las herramientas. \
Pruebas las tres herramientas de sales_agent una vez antes de elegir la mejor. \
Eliges el mejor correo electrónico y usas la herramienta send_email para enviar el mejor correo electrónico (y solo el mejor) al usuario."


sales_manager = Agent(name="Manager de ventas", instructions=instructions, tools=tools, model=gemini_model_agent1)

message = "Envía un correo electrónico de ventas frío dirigido a 'Estimado director ejecutivo'"

with trace("Manager de ventas"):
    result = await Runner.run(sales_manager, message)

### Hand-Off : Transferencia o delegación de Trabajo
Agregamos ahora el concepto de `Hands-off` con el cual se le da la facilidad a un modelo llm  que utilice el conocimiento de otro modelo llm con esto el flujo de trabajo de la solución dejaría de ser un `workflow` en donde un agente usa herramientas a un flujo `agentic` donde hay una conversacion entre varios agentes (según Antrophic)

In [33]:
subject_instructions = "Puedes escribir un asunto para un correo electrónico de ventas en frío. \
    Se te proporciona un mensaje y necesitas escribir un asunto para un correo electrónico que probablemente obtenga respuesta."

html_instructions = "Puedes convertir un cuerpo de correo electrónico de texto a un cuerpo de correo electrónico HTML. \
    Se te proporciona un cuerpo de correo electrónico de texto que puede tener algún markdown \
    y necesitas convertirlo a un cuerpo de correo electrónico HTML con un diseño simple, claro y atractivo."

subject_writer = Agent(name="Escritor de asunto de correo electrónico", instructions=subject_instructions, model=gemini_model_agent1)
subject_tool = subject_writer.as_tool(tool_name="subject_writer", 
                                      tool_description="Escribe un asunto para un correo electrónico de ventas en frío")

#herramientas para mejorar el correo                                      
html_converter = Agent(name="Conversor de cuerpo de correo electrónico HTML", instructions=html_instructions, model=gemini_model_agent1)
html_tool = html_converter.as_tool(tool_name="html_converter",
                                   tool_description="Convierte un cuerpo de correo electrónico de texto a un cuerpo de correo electrónico HTML")

In [34]:
#nueva herramienta que envia el correo html con sendgrid
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Envía un correo electrónico con el asunto y el cuerpo HTML a todos los clientes potenciales de ventas """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("ing.cristian.roac@gmail.com")  # Cambiar a tu remitente verificado
    to_email = To("ing.cristian.roac@gmail.com")  # Cambiar a sureceptor
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

tools = [subject_tool, html_tool, send_html_email]

In [35]:
#creamos un Nuevo Agente para el hands-off 

instructions ="Eres un formateador y remitente de correos electrónicos. \
    Recibes el cuerpo de un correo electrónico para enviarlo. \
    Primero usas la herramienta subject_writer para escribir un asunto para el correo electrónico, \
    luego usas la herramienta html_converter para convertir el cuerpo a HTML. \
    Finalmente, usas la herramienta send_html_email para enviar el correo electrónico con el asunto y el cuerpo HTML."


emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=tools,
    model=gemini_model_agent2,
    handoff_description="Convierte un email a HTML y lo envía")

In [36]:

handoffs = [emailer_agent]


In [39]:
sales_manager_instructions = "Eres un gerente de ventas que trabaja para Fivok Cloud Soluctions. Utilizas las herramientas que se te proporcionan para generar correos electrónicos de ventas en frío. \
Nunca generas correos electrónicos de ventas tú mismo; siempre utilizas las herramientas. \
Pruebas las 3 herramientas del agente de ventas al menos una vez antes de elegir la mejor. \
Puedes usar las herramientas múltiples veces si no estás satisfecho con los resultados del primer intento. \
Seleccionas el mejor correo electrónico usando tu propio criterio sobre cuál será más efectivo. \
Después de elegir el correo electrónico, transfieres al agente Email Manager para formatear y enviar el correo."


sales_manager = Agent(
    name="Manager de ventas",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model=gemini_model_agent2)

message = "Envía un correo electrónico de ventas en frío dirigido a 'Estimado director ejecutivo'"

with trace("Automated SDR"):
    result = await Runner.run(sales_manager, message)

### Guarda Railes
Un `guardrail` es un agente o funcion que se encarga de monitorear los resultados de los demas agentes con el fin de `catchear` un comportamiento inadecuado que definamos desde el dominio de nuestra apliacion, cuando el guarda rail se activa, este ejecuta una excepcion que puede ser utilizada para aplicar logica de negocio

In [42]:
class NameCheckOutput(BaseModel):
    is_name_in_message: bool
    name: str

guardrail_agent = Agent( 
    name="Revisión de nombre",
    instructions="Revisa si el usuario está incluyendo el nombre personal de alguien en lo que quiere que hagas.",
    output_type=NameCheckOutput,
    model=gemini_model_agent1
)

In [43]:
#se define el guardrail mediante la anotacion @input_guardrail
@input_guardrail
async def guardrail_against_name(ctx, agent, message):
    result = await Runner.run(guardrail_agent, message, context=ctx.context)
    is_name_in_message = result.final_output.is_name_in_message
    return GuardrailFunctionOutput(output_info={"found_name": result.final_output},tripwire_triggered=is_name_in_message)

In [44]:
careful_sales_manager = Agent(
    name="Manager de ventas",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=[emailer_agent],
    model=gemini_model_agent2,
    input_guardrails=[guardrail_against_name]
    )

message = "Envía un correo electrónico de ventas en frío dirigido a 'Estimado director ejecutivo' desde Alice"

with trace("Protected Automated SDR"):
    result = await Runner.run(careful_sales_manager, message)

InputGuardrailTripwireTriggered: Guardrail InputGuardrail triggered tripwire

Se observa la excepción activada por el guarda rail, cuya función era detectar si el correo electrónico se envía desde un NOMBRE 

In [ ]:
 model_settings=ModelSettings(tool_choice="required") # esta linea garantiza que el agente utilice obligatoriamente la tool definida
 #ejemplo
 search_agent = Agent(
    name="Agente de búsqueda",
    instructions=INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")],#hosttools herramienta alojada por openia
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required"),
)